<a href="https://colab.research.google.com/github/MathewNhari/CV/blob/master/Final_Data_Cleaning_Decision_Tree_793_49.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#**Importing Data**

In [ ]:
riders = pd.read_csv('Riders.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')
Test_df = pd.read_csv('Test.csv')
Train_df = pd.read_csv('Train.csv', parse_dates= ['Placement - Day of Month'])
VariableDefinitions = pd.read_csv('VariableDefinitions.csv')

In [ ]:
VariableDefinitions

,Order No,Unique number identifying the order
0,User Id,Unique number identifying the customer on a pl...
1,Vehicle Type,"For this competition limited to bikes, however..."
2,Platform Type,"Platform used to place the order, there are 4 ..."
3,Personal or Business,Customer type
4,Placement - Day of Month,Placement - Day of Month i.e 1-31
5,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
6,Placement - Time,Placement - Time - Time of day the order was p...
7,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
8,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)
9,Confirmation - Time,Confirmation - Time - Time of day the order wa...


**Sample Submission**

In [ ]:
SampleSubmission.head(1)

,Order_No,Time from Pickup to Arrival
0,Order_No_19248,567.0


In [ ]:
riders.head(1)

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159


In [ ]:
Train_df.head(1)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.83037,-1.300406,36.829741,Rider_Id_432,745


In [ ]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  object 
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

In [ ]:
#Train_df = Train_df.drop(['Precipitation in millimeters'], axis= 1)
#Test_df = Test_df.drop(['Precipitation in millimeters'], axis= 1)

# Joining Tables

In [ ]:
Train_df = Train_df.merge(riders, on='Rider Id', how='left')
Test_df = Test_df.merge(riders, on='Rider Id', how='left')

# **Data Cleaning**

**Checking for Duplicates**

In [ ]:
#No duplicate Data On Test Data
def Check_duplicates(df):
  duplicates = df.duplicated(subset = ['Order No'], keep = False)
  return df[duplicates]

In [ ]:
#No duplicates 
Train_duplicates = Check_duplicates(Train_df)
Test_duplucates = Check_duplicates(Test_df)

**Changing into int data types

In [ ]:
Train_df['Platform Type'] = Train_df['Platform Type'].astype('object')
Test_df['Platform Type'] = Test_df['Platform Type'].astype('object')       
Train_df['Placement - Day of Month'] = Train_df['Placement - Day of Month'].astype('int')
Test_df['Placement - Day of Month'] = Test_df['Placement - Day of Month'].astype('int')

**Dropping columns not in test Data**

In [ ]:
def columnDrop(df,columns):
  df.drop(columns, axis=1,inplace=True)
  #df.set_index(['Order No'], inplace= True)
  return df

In [ ]:
Drop_Train_Cols = ['Precipitation in millimeters', 'Arrival at Destination - Day of Month', 
                   'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Time']
Drop_Test_Cols =  ['Precipitation in millimeters']    
Test_df = columnDrop(Test_df,Drop_Test_Cols)
Train_df = columnDrop(Train_df,Drop_Train_Cols)

**Seperating Time**

In [ ]:
def timeconvert(df,Time):
    df[Time] = df[Time].astype('datetime64')
    return df

In [ ]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train_df = timeconvert(Train_df,Time_columns)
Test_df = timeconvert(Test_df,Time_columns)

In [ ]:
def SplitDatetime(df,columns):
  for item in columns:
    #df[item+'_day'] = pd.to_datetime(df[item]).dt.day
    df[item+'_hour'] = pd.to_datetime(df[item]).dt.hour
    df[item+'_minutes'] = pd.to_datetime(df[item]).dt.minute
    df[item+'_seconds'] = pd.to_datetime(df[item]).dt.second
  df.drop(columns,axis=1, inplace = True)
  return df

In [ ]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train_df = SplitDatetime(Train_df,Time_columns)
Test_df = SplitDatetime(Test_df,Time_columns)

In [ ]:
Train_df.head(1)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9,5,9,5,9,5,4,20.4,-1.317755,36.83037,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549,9,35,46,9,40,10,10,4,47,10,27,30


**Taking Care of missing Data

In [ ]:
from sklearn.impute import SimpleImputer

def missingData(df,columns):
  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
  imputer.fit(df[columns])
  df[columns] = imputer.transform(df[columns])
  return df

In [ ]:
columns = ['Temperature']
Test_df = missingData(Test_df,columns)
Train_df = missingData(Train_df,columns)

In [ ]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Order No                              21201 non-null  object 
 1   User Id                               21201 non-null  object 
 2   Vehicle Type                          21201 non-null  object 
 3   Platform Type                         21201 non-null  object 
 4   Personal or Business                  21201 non-null  object 
 5   Placement - Day of Month              21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)          21201 non-null  int64  
 7   Confirmation - Day of Month           21201 non-null  int64  
 8   Confirmation - Weekday (Mo = 1)       21201 non-null  int64  
 9   Arrival at Pickup - Day of Month      21201 non-null  int64  
 10  Arrival at Pickup - Weekday (Mo = 1)  21201 non-null  int64  
 11  Pickup - Day of

# **Feature Engineering**

**Get Dummy Variables**

In [ ]:
def dummy_encoder(df, cols):
  dummies = pd.get_dummies(df,columns= cols,drop_first = True)
  merged = pd.concat([df,dummies],axis='columns')
  final = merged.drop(cols, axis = 1)
  return dummies

In [ ]:
cols = ['Platform Type','Personal or Business', 'Placement - Weekday (Mo = 1)',
        'Confirmation - Weekday (Mo = 1)','Pickup - Weekday (Mo = 1)','Rider Id','Vehicle Type','User Id']
Train_df = dummy_encoder(Train_df, cols)
Test_df = dummy_encoder(Test_df, cols)
Test_df.head(1)

,Order No,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Placement - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_3,Placement - Weekday (Mo = 1)_4,Placement - Weekday (Mo = 1)_5,Placement - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_7,Confirmation - Weekday (Mo = 1)_2,Confirmation - Weekday (Mo = 1)_3,...,User Id_User_Id_909,User Id_User_Id_910,User Id_User_Id_912,User Id_User_Id_913,User Id_User_Id_914,User Id_User_Id_915,User Id_User_Id_917,User Id_User_Id_918,User Id_User_Id_919,User Id_User_Id_920,User Id_User_Id_922,User Id_User_Id_924,User Id_User_Id_925,User Id_User_Id_928,User Id_User_Id_93,User Id_User_Id_930,User Id_User_Id_935,User Id_User_Id_937,User Id_User_Id_938,User Id_User_Id_94,User Id_User_Id_941,User Id_User_Id_944,User Id_User_Id_948,User Id_User_Id_95,User Id_User_Id_953,User Id_User_Id_954,User Id_User_Id_955,User Id_User_Id_957,User Id_User_Id_969,User Id_User_Id_97,User Id_User_Id_970,User Id_User_Id_972,User Id_User_Id_973,User Id_User_Id_978,User Id_User_Id_980,User Id_User_Id_981,User Id_User_Id_991,User Id_User_Id_992,User Id_User_Id_995,User Id_User_Id_996
0,Order_No_19248,27,27,27,3,27,8,23.24612,-1.333275,36.870815,-1.305249,36.82239,439,1511,13.3,171,16,44,10,16,44,29,16,53,4,17,6,47,0,1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Train_df.head(1)

,Order No,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Placement - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_3,Placement - Weekday (Mo = 1)_4,Placement - Weekday (Mo = 1)_5,Placement - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_7,Confirmation - Weekday (Mo = 1)_2,...,User Id_User_Id_96,User Id_User_Id_960,User Id_User_Id_961,User Id_User_Id_962,User Id_User_Id_963,User Id_User_Id_964,User Id_User_Id_965,User Id_User_Id_966,User Id_User_Id_967,User Id_User_Id_968,User Id_User_Id_969,User Id_User_Id_97,User Id_User_Id_970,User Id_User_Id_971,User Id_User_Id_972,User Id_User_Id_974,User Id_User_Id_975,User Id_User_Id_976,User Id_User_Id_977,User Id_User_Id_978,User Id_User_Id_979,User Id_User_Id_98,User Id_User_Id_981,User Id_User_Id_982,User Id_User_Id_983,User Id_User_Id_984,User Id_User_Id_985,User Id_User_Id_986,User Id_User_Id_987,User Id_User_Id_988,User Id_User_Id_989,User Id_User_Id_99,User Id_User_Id_990,User Id_User_Id_991,User Id_User_Id_993,User Id_User_Id_994,User Id_User_Id_995,User Id_User_Id_997,User Id_User_Id_998,User Id_User_Id_999
0,Order_No_4211,9,9,9,5,9,4,20.4,-1.317755,36.83037,-1.300406,36.829741,745,1637,1309,13.8,549,9,35,46,9,40,10,10,4,47,10,27,30,0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Variable Selection by Correlation and Significance**

In [ ]:
Train = Train_df.drop(['Order No'],axis=1)

In [ ]:
# Calculate correlations between predictor variables and the response variable
corrs = Train.corr()['Time from Pickup to Arrival'].sort_values(ascending=False)

In [ ]:
from scipy.stats import pearsonr

# Build a dictionary of correlation coefficients and p-values
dict_cp = {}

column_titles = [col for col in corrs.index if col!= 'Time from Pickup to Arrival']
for col in column_titles:
    p_val = round(pearsonr(Train[col], Train['Time from Pickup to Arrival'])[1],6)
    dict_cp[col] = {'Correlation_Coefficient':corrs[col],
                    'P_Value':p_val}

df_cp = pd.DataFrame(dict_cp).T
df_cp_sorted = df_cp.sort_values('P_Value')
df_cp_sorted[df_cp_sorted['P_Value']<0.1]

,Correlation_Coefficient,P_Value
Distance (KM),0.580608,0.000000
Rider Id_Rider_Id_576,-0.037443,0.000000
No_Of_Orders,-0.041608,0.000000
User Id_User_Id_3149,-0.041799,0.000000
Average_Rating,-0.042559,0.000000
...,...,...
Rider Id_Rider_Id_168,0.011351,0.098385
Rider Id_Rider_Id_333,-0.011328,0.099070
Rider Id_Rider_Id_946,0.011321,0.099278
Rider Id_Rider_Id_530,0.011312,0.099552


In [ ]:
# The dependent variable remains the same:
y_data = Train['Time from Pickup to Arrival'] 

# Model building - Independent Variable (IV) DataFrame
X_names = list(df_cp[df_cp['P_Value'] < 0.05].index)
X_data = Train[X_names]

In [ ]:
X_data.head()

,Distance (KM),Destination Long,Rider Id_Rider_Id_519,User Id_User_Id_2982,Pickup Long,Rider Id_Rider_Id_831,User Id_User_Id_1271,User Id_User_Id_1245,User Id_User_Id_3599,User Id_User_Id_136,User Id_User_Id_658,User Id_User_Id_552,Rider Id_Rider_Id_268,User Id_User_Id_1927,User Id_User_Id_2018,Rider Id_Rider_Id_697,Rider Id_Rider_Id_883,User Id_User_Id_3262,Rider Id_Rider_Id_554,User Id_User_Id_2931,Rider Id_Rider_Id_447,Rider Id_Rider_Id_155,Rider Id_Rider_Id_418,User Id_User_Id_3547,Rider Id_Rider_Id_498,User Id_User_Id_84,User Id_User_Id_2132,User Id_User_Id_3434,Rider Id_Rider_Id_680,User Id_User_Id_1384,Rider Id_Rider_Id_64,Rider Id_Rider_Id_332,Rider Id_Rider_Id_787,User Id_User_Id_3238,User Id_User_Id_187,Rider Id_Rider_Id_244,User Id_User_Id_842,User Id_User_Id_3586,User Id_User_Id_1051,User Id_User_Id_1179,...,Rider Id_Rider_Id_179,Placement - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_6,Confirmation - Weekday (Mo = 1)_6,Rider Id_Rider_Id_427,Rider Id_Rider_Id_855,Rider Id_Rider_Id_953,User Id_User_Id_1527,Rider Id_Rider_Id_613,Rider Id_Rider_Id_726,Rider Id_Rider_Id_927,User Id_User_Id_53,User Id_User_Id_3557,Rider Id_Rider_Id_291,Rider Id_Rider_Id_70,User Id_User_Id_1209,User Id_User_Id_1363,Rider Id_Rider_Id_41,User Id_User_Id_3201,Rider Id_Rider_Id_137,Rider Id_Rider_Id_472,Rider Id_Rider_Id_576,No_Of_Orders,User Id_User_Id_3149,Average_Rating,User Id_User_Id_3291,User Id_User_Id_1500,User Id_User_Id_517,Rider Id_Rider_Id_239,Rider Id_Rider_Id_51,User Id_User_Id_3283,Rider Id_Rider_Id_619,Rider Id_Rider_Id_10,User Id_User_Id_868,Rider Id_Rider_Id_39,Pickup Lat,Rider Id_Rider_Id_515,Destination Lat,User Id_User_Id_633,User Id_User_Id_393
0,4,36.829741,0,0,36.830370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1637,0,13.8,0,0,0,0,0,0,0,0,0,0,-1.317755,0,-1.300406,1,0
1,16,36.814358,0,0,36.899315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,396,0,13.6,0,0,0,0,0,0,0,0,0,0,-1.351453,0,-1.295004,0,0
2,3,36.828195,0,0,36.843419,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1023,0,12.5,0,0,0,0,0,0,0,0,0,0,-1.308284,0,-1.300921,0,0
3,9,36.795063,0,0,36.832396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,886,0,14.5,0,0,0,0,0,0,0,0,0,0,-1.281301,0,-1.257147,0,0
4,9,36.809817,0,0,36.792118,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2311,0,14.1,0,0,0,0,0,0,0,0,0,0,-1.266597,0,-1.295041,0,0


In [ ]:
# Create the correlation matrix
corr = X_data.corr()

# Find rows and columns where correlation coefficients > 0.9 or <-0.9
corr[np.abs(corr) > 0.9]

,Distance (KM),Destination Long,Rider Id_Rider_Id_519,User Id_User_Id_2982,Pickup Long,Rider Id_Rider_Id_831,User Id_User_Id_1271,User Id_User_Id_1245,User Id_User_Id_3599,User Id_User_Id_136,User Id_User_Id_658,User Id_User_Id_552,Rider Id_Rider_Id_268,User Id_User_Id_1927,User Id_User_Id_2018,Rider Id_Rider_Id_697,Rider Id_Rider_Id_883,User Id_User_Id_3262,Rider Id_Rider_Id_554,User Id_User_Id_2931,Rider Id_Rider_Id_447,Rider Id_Rider_Id_155,Rider Id_Rider_Id_418,User Id_User_Id_3547,Rider Id_Rider_Id_498,User Id_User_Id_84,User Id_User_Id_2132,User Id_User_Id_3434,Rider Id_Rider_Id_680,User Id_User_Id_1384,Rider Id_Rider_Id_64,Rider Id_Rider_Id_332,Rider Id_Rider_Id_787,User Id_User_Id_3238,User Id_User_Id_187,Rider Id_Rider_Id_244,User Id_User_Id_842,User Id_User_Id_3586,User Id_User_Id_1051,User Id_User_Id_1179,...,Rider Id_Rider_Id_179,Placement - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_6,Confirmation - Weekday (Mo = 1)_6,Rider Id_Rider_Id_427,Rider Id_Rider_Id_855,Rider Id_Rider_Id_953,User Id_User_Id_1527,Rider Id_Rider_Id_613,Rider Id_Rider_Id_726,Rider Id_Rider_Id_927,User Id_User_Id_53,User Id_User_Id_3557,Rider Id_Rider_Id_291,Rider Id_Rider_Id_70,User Id_User_Id_1209,User Id_User_Id_1363,Rider Id_Rider_Id_41,User Id_User_Id_3201,Rider Id_Rider_Id_137,Rider Id_Rider_Id_472,Rider Id_Rider_Id_576,No_Of_Orders,User Id_User_Id_3149,Average_Rating,User Id_User_Id_3291,User Id_User_Id_1500,User Id_User_Id_517,Rider Id_Rider_Id_239,Rider Id_Rider_Id_51,User Id_User_Id_3283,Rider Id_Rider_Id_619,Rider Id_Rider_Id_10,User Id_User_Id_868,Rider Id_Rider_Id_39,Pickup Lat,Rider Id_Rider_Id_515,Destination Lat,User Id_User_Id_633,User Id_User_Id_393
Distance (KM),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Destination Long,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rider Id_Rider_Id_519,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User Id_User_Id_2982,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pickup Long,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pickup Lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
# As before, we create the correlation matrix
# and find rows and columnd where correlation coefficients > 0.9 or <-0.9
corr = X_data.corr()
r, c = np.where(np.abs(corr) > 0.9)

# We are only interested in the off diagonal entries:
off_diagonal = np.where(r != c)

# Show the correlation matrix rows and columns where we have highly correlated off diagonal entries:
corr.iloc[r[off_diagonal], c[off_diagonal]]

,Placement - Weekday (Mo = 1)_5,Confirmation - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_5,Confirmation - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_5,Placement - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_2,Confirmation - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_2,Confirmation - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_2,Rider Id_Rider_Id_83,User Id_User_Id_2270,Arrival at Pickup - Day of Month,Pickup - Day of Month,Placement - Day of Month,Confirmation - Day of Month,Pickup - Day of Month,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Pickup - Day of Month,Pickup - Weekday (Mo = 1)_6,Confirmation - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_6,Confirmation - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_6
Pickup - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Pickup - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Placement - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Placement - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Confirmation - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Confirmation - Weekday (Mo = 1)_5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.003308,-0.003308,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008841,0.008833,0.008833,0.008833,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185,-0.119185
Confirmation - Weekday (Mo = 1)_2,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,1.000000,0.999689,1.000000,0.999689,1.000000,1.000000,-0.003291,-0.003291,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004133,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559
Confirmation - Weekday (Mo = 1)_2,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,1.000000,0.999689,1.000000,0.999689,1.000000,1.000000,-0.003291,-0.003291,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004133,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559
Pickup - Weekday (Mo = 1)_2,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,-0.230825,1.000000,0.999689,1.000000,0.999689,1.000000,1.000000,-0.003291,-0.003291,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004127,0.004133,0.004133,0.004133,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559,-0.118559
Pickup - Weekday (Mo = 1)_2,-0.230825,-0.230825,-0.230825,-0.230825,-0.2308

In [ ]:
DropColumns = ['Pickup - Weekday (Mo = 1)_5', 'Placement - Weekday (Mo = 1)_5','Placement - Weekday (Mo = 1)_5',
               'Confirmation - Weekday (Mo = 1)_5', 'Confirmation - Weekday (Mo = 1)_5', 'Confirmation - Weekday (Mo = 1)_2',
               'Pickup - Weekday (Mo = 1)_2','Pickup - Weekday (Mo = 1)_2', 'Placement - Weekday (Mo = 1)_2',
               'Placement - Weekday (Mo = 1)_2','User Id_User_Id_2270','Rider Id_Rider_Id_83','Confirmation - Day of Month','Confirmation - Day of Month',
               'Confirmation - Day of Month','Arrival at Pickup - Day of Month','Arrival at Pickup - Day of Month',
               'Arrival at Pickup - Day of Month','Pickup - Day of Month','Pickup - Day of Month','Pickup - Day of Month',
               'Placement - Day of Month','Placement - Day of Month','Placement - Day of Month','Placement - Weekday (Mo = 1)_6',
               'Pickup - Weekday (Mo = 1)_6','Pickup - Weekday (Mo = 1)_6','Confirmation - Weekday (Mo = 1)_6',
               'Confirmation - Weekday (Mo = 1)_6']

In [ ]:
X_data = X_data.drop(DropColumns,axis=1)

# Building SVR Model


**Feature Scaling**

In [ ]:
# import the scaling module
##from sklearn.preprocessing import StandardScaler

In [ ]:
# create standardization object
##scaler_X = StandardScaler()
##scaler_y = StandardScaler()

In [ ]:
##columns = ['Distance (KM)', 'Destination Long','Pickup Long']
# save standardized features into new variable
##X_data[columns] = scaler_X.fit_transform(X_data[columns])
##y_data = scaler_y.fit_transform(np.array(y_data).reshape(-1,1))
#y_data = np.array(y_data).reshape(-1,1)


**splitting features into test set and training set**

In [ ]:
# import train/test split module
from sklearn.model_selection import train_test_split

In [ ]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    test_size=0.20,
                                                    random_state=1)

**Dicision Tree Model**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
reg_tree = DecisionTreeRegressor(max_depth=3)

In [ ]:
reg_tree.fit(X_train,y_train)
y_pred = reg_tree.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
DT_rmse = mean_squared_error(y_test, y_pred, squared= False)

In [ ]:
DT_rmse

793.4983953114167